In [ ]:
#by Tuba Gök
import cv2
import numpy as np
import os

# Initialize the cameras
cap_left = cv2.VideoCapture(0)
cap_right = cv2.VideoCapture(1)

# Create folders to save captured images
left_folder = 'left_images'
right_folder = 'right_images'
os.makedirs(left_folder, exist_ok=True)
os.makedirs(right_folder, exist_ok=True)

def capture_images():
    ret_left, frame_left = cap_left.read()
    ret_right, frame_right = cap_right.read()
    if ret_left and ret_right:
        return frame_left, frame_right
    else:
        return None, None

image_count = 0

print("Press 's' to save the frame, 'c' to skip, and 'q' to quit.")

while True:
    frame_left, frame_right = capture_images()
    if frame_left is not None and frame_right is not None:
        cv2.imshow('Left Camera', frame_left)
        cv2.imshow('Right Camera', frame_right)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            cv2.imwrite(os.path.join(left_folder,
f'left_{image_count}.png'), frame_left)
            cv2.imwrite(os.path.join(right_folder,
f'right_{image_count}.png'), frame_right)
            print(f"Saved frame {image_count}")
            image_count += 1
        elif key == ord('c'):
            print("Frame skipped")
        elif key == ord('q'):
            break
    else:
        print("Failed to capture images from both cameras")

# Release the cameras and close windows
cap_left.release()
cap_right.release()
cv2.destroyAllWindows()

In [ ]:
# Calibration
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((8*6, 3), np.float32)
objp[:, :2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2) * 25  # 25mm square size
objpoints = []
imgpoints_left = []
imgpoints_right = []

left_images = sorted([os.path.join(left_folder, f) for f in
os.listdir(left_folder) if f.endswith('.png')])
right_images = sorted([os.path.join(right_folder, f) for f in
os.listdir(right_folder) if f.endswith('.png')])

for left_img_path, right_img_path in zip(left_images, right_images):
    img_left = cv2.imread(left_img_path)
    img_right = cv2.imread(right_img_path)

    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    ret_left, corners_left = cv2.findChessboardCorners(gray_left, (8, 6), None)
    ret_right, corners_right = cv2.findChessboardCorners(gray_right,
(8, 6), None)

    if ret_left and ret_right:
        objpoints.append(objp)

        corners_left = cv2.cornerSubPix(gray_left, corners_left, (11,
11), (-1, -1), criteria)
        imgpoints_left.append(corners_left)

        corners_right = cv2.cornerSubPix(gray_right, corners_right,
(11, 11), (-1, -1), criteria)
        imgpoints_right.append(corners_right)

        cv2.drawChessboardCorners(img_left, (8, 6), corners_left, ret_left)
        cv2.drawChessboardCorners(img_right, (8, 6), corners_right, ret_right)
        cv2.imshow('img_left', img_left)
        cv2.imshow('img_right', img_right)
        cv2.waitKey(500)

cv2.destroyAllWindows()

ret_left, mtx_left, dist_left, rvecs_left, tvecs_left =
cv2.calibrateCamera(objpoints, imgpoints_left, gray_left.shape[::-1],
None, None)
ret_right, mtx_right, dist_right, rvecs_right, tvecs_right =
cv2.calibrateCamera(objpoints, imgpoints_right,
gray_right.shape[::-1], None, None)

ret, mtx_left, dist_left, mtx_right, dist_right, R, T, E, F =
cv2.stereoCalibrate(
    objpoints, imgpoints_left, imgpoints_right,
    mtx_left, dist_left, mtx_right, dist_right,
    gray_left.shape[::-1], criteria, flags=cv2.CALIB_FIX_INTRINSIC
)


In [ ]:
#rectification


R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    mtx_left, dist_left, mtx_right, dist_right, gray_left.shape[::-1],
R, T, alpha=0
)

map_left_x, map_left_y = cv2.initUndistortRectifyMap(mtx_left,
dist_left, R1, P1, gray_left.shape[::-1], cv2.CV_16SC2)
map_right_x, map_right_y = cv2.initUndistortRectifyMap(mtx_right,
dist_right, R2, P2, gray_right.shape[::-1], cv2.CV_16SC2)

img_left = cv2.imread(left_images[0])
img_right = cv2.imread(right_images[0])

rectified_left = cv2.remap(img_left, map_left_x, map_left_y, cv2.INTER_LINEAR)
rectified_right = cv2.remap(img_right, map_right_x, map_right_y,
cv2.INTER_LINEAR)

cv2.imshow('rectified_left', rectified_left)
cv2.imshow('rectified_right', rectified_right)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#calculating disparity map

def compute_disparity_map(img_left, img_right):
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    gray_left = cv2.GaussianBlur(gray_left, (5, 5), 0)
    gray_right = cv2.GaussianBlur(gray_right, (5, 5), 0)

    window_size = 5
    min_disp = 0
    num_disp = 16 * 12

    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=window_size,
        P1=8 * 3 * window_size ** 2,
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=50,
        speckleRange=2,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )

    disparity = stereo.compute(gray_left, gray_right).astype(np.float32) / 16.0

    return disparity, min_disp, num_disp

In [ ]:
# calculating depth and measuring the distance

def calculate_depth(disparity, Q):
    epsilon = 1e-6
    disparity[disparity <= 0] = epsilon

    depth = cv2.reprojectImageTo3D(disparity, Q)[:, :, 2]

    depth[depth > 10000] = np.nan

    return depth

def measure_distance(depth, mask=None):
    if mask is not None:
        depth = np.where(mask, depth, np.nan)

    finite_depth = depth[np.isfinite(depth)]
    if finite_depth.size == 0:
        return np.inf
    avg_distance = np.mean(finite_depth)
    return avg_distance

cap_left = cv2.VideoCapture(0)
cap_right = cv2.VideoCapture(1)

print("Press 'q' to quit")

ret, bg_frame_left = cap_left.read()
bg_frame_left_gray = cv2.cvtColor(bg_frame_left, cv2.COLOR_BGR2GRAY)

while True:
    ret_left, frame_left = cap_left.read()
    ret_right, frame_right = cap_right.read()

    if ret_left and ret_right:
        rectified_left = cv2.remap(frame_left, map_left_x, map_left_y,
cv2.INTER_LINEAR)
        rectified_right = cv2.remap(frame_right, map_right_x,
map_right_y, cv2.INTER_LINEAR)

        disparity, min_disp, num_disp =
compute_disparity_map(rectified_left, rectified_right)

        depth = calculate_depth(disparity, Q)

        rectified_left_gray = cv2.cvtColor(rectified_left, cv2.COLOR_BGR2GRAY)
        diff_frame = cv2.absdiff(rectified_left_gray, bg_frame_left_gray)
        _, mask = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)

        distance = measure_distance(depth, mask) * (0.087)
        if np.isfinite(distance):
            print(f"Measured distance: {distance:.2f} centimeters")
        else:
            print("Distance measurement out of bounds")

        cv2.imshow('rectified_left', rectified_left)
        cv2.imshow('rectified_right', rectified_right)
        cv2.imshow('disparity', (disparity - min_disp) / num_disp)
        cv2.imshow('mask', mask.astype(np.uint8) * 255)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print("Failed to capture images from both cameras")

cap_left.release()
cap_right.release()
cv2.destroyAllWindows()
